In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import xarray as xr
import math
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sys.path.append('/home/cucchi/phd/devel/phd/')
RESP_PATH = '/home/cucchi/phd/data/response/lorenz96/rk4/SF_8_1.0_0/'
obs = 'energy_bin_0.98q_0.99q'
linear_coefficient = 0.01
activation_time = 0
deactivation_time = 100
forcing = 'LF_8_{}_{}_{}'.format(
    linear_coefficient,
    activation_time,
    deactivation_time
)

In [ ]:
from lab import analysis
from lab.simulation import forcings

Average response

In [ ]:
response_p1 = xr.open_dataarray(os.path.join(RESP_PATH, 'response_lorenz96_rk4_{}_SF_8_1.0_0.nc'.format(obs)))
plt.plot(response_p1)

Susceptibility

In [ ]:
chi = analysis.compute_susceptibility(response_p1.values.squeeze())
plt.plot(chi)

Predicted Response

In [ ]:
resp_pred = analysis.compute_response(chi, forcings.LinearForcing(
    linear_coefficient=linear_coefficient,
    activation_time=activation_time,
    deactivation_time=deactivation_time
))
plt.plot(resp_pred[0:10000])

Real Response

In [ ]:
resp_real = xr.open_dataarray('/home/cucchi/phd/data/response/lorenz96/rk4/{f}/response_lorenz96_rk4_{o}_{f}.nc'.format(
    o=obs,
    f=forcing
))
resp_real.plot()

Comparison

In [ ]:
plt.plot(resp_real[0:10000])
plt.plot(resp_pred[0:10000])

### Average

In [ ]:
std_pred = resp_pred[0:10000].std()
sig_digit_pred = -math.floor(math.log10(std_pred))
print('predicted: {} +/- {}'.format(
    round(resp_pred[0:10000].mean(), sig_digit_pred), 
    round(resp_pred[0:10000].std(),sig_digit_pred)
))
std_real = resp_real.std().values
sig_digit_real = -math.floor(math.log10(std_real))
print('real:      {} +/- {}'.format(
    round(float(resp_real.mean().values), sig_digit_real), 
    round(float(resp_real.std().values), sig_digit_real)
))

### Block-Average

In [ ]:
resp_real.rolling(time_step=1000, center=True).mean().plot()
plt.plot(resp_pred[0:10000])

In [ ]:
resp_real.rolling(time_step=100, center=True).mean().plot()
plt.plot(resp_pred[0:10000])

In [ ]:
resp_real.rolling(time_step=10, center=True).mean().plot()
plt.plot(resp_pred[0:10000])

## Semi-Difference

In [ ]:
RESP_PATH = '/home/cucchi/phd/data/response/lorenz96/rk4/SF_8_-1.0_0/'
response_m1 = xr.open_dataarray(os.path.join(RESP_PATH, 'response_lorenz96_rk4_{}_SF_8_-1.0_0.nc'.format(obs)))
response = (response_p1 - response_m1)/2
chi = analysis.compute_susceptibility(response.values.squeeze())
resp_pred = analysis.compute_response(chi, forcings.LinearForcing(
    linear_coefficient=linear_coefficient,
    activation_time=activation_time,
    deactivation_time=deactivation_time
))
plt.plot(resp_pred[0:10000])

In [ ]:
plt.plot(resp_real[0:10000])
plt.plot(resp_pred[0:10000])

### Average

In [ ]:
std_pred = resp_pred[0:10000].std()
sig_digit_pred = -math.floor(math.log10(std_pred))
print('predicted: {} +/- {}'.format(
    round(resp_pred[0:10000].mean(), sig_digit_pred), 
    round(resp_pred[0:10000].std(),sig_digit_pred)
))
std_real = resp_real.std().values
sig_digit_real = -math.floor(math.log10(std_real))
print('real:      {} +/- {}'.format(
    round(float(resp_real.mean().values), sig_digit_real), 
    round(float(resp_real.std().values), sig_digit_real)
))

### Block-Average

In [ ]:
resp_real.rolling(time_step=1000, center=True).mean().plot()
plt.plot(resp_pred[0:10000])

In [ ]:
resp_real.rolling(time_step=100, center=True).mean().plot()
plt.plot(resp_pred[0:10000])

In [ ]:
resp_real.rolling(time_step=10, center=True).mean().plot()
plt.plot(resp_pred[0:10000])